In [10]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import wkt
from shapely.ops import nearest_points
import dask.dataframe as dd
import geopandas as gpd
import h3
import pandas as pd
from shapely import wkt
import pytz

In [26]:
%%time
schools = gpd.read_file("/Users/catherinehayden/WB/cluj/Cluj_geospatial data/Schools_ClujNapoca.shp")
hospitals = gpd.read_file("/Users/catherinehayden/WB/cluj/Cluj_geospatial data/Hospitals_ClujNapoca.shp")

CPU times: user 40.5 ms, sys: 4.61 ms, total: 45.2 ms
Wall time: 51.3 ms


In [27]:
%%time
output = "/Users/catherinehayden/WB/cluj/streetcsvs"

roads = gpd.read_file("/Users/catherinehayden/WB/cluj/Cluj_geospatial data/RoadNetwork_ClujNapoca.shp")
schools = gpd.read_file("/Users/catherinehayden/WB/cluj/Cluj_geospatial data/Schools_ClujNapoca.shp")
hospitals = gpd.read_file("/Users/catherinehayden/WB/cluj/Cluj_geospatial data/Hospitals_ClujNapoca.shp")
gdf =  gpd.read_file('geo.csv')
#roads = roads[['NAME', 'COUNTY', 'Length', 'geometry']]
roads = roads.rename(columns = {'NAME': 'streetname'})

CPU times: user 22 s, sys: 1.02 s, total: 23 s
Wall time: 23.3 s


In [29]:
%%time

# add desired path for data storage here
location =  "/Users/catherinehayden/WB/cluj" #where on your system do you want clean.csv?
output_path = location + "/clean.csv"

# read data into Dask
PATH = "RO_9clujnapoca_alerts.000000000000.csv.gz"
ddf = dd.read_csv(
        PATH, 
        compression="gzip", 
        blocksize=None, 
        dtype={
            'confidence': 'int8',
            'type': 'category',
            'subtype': 'category',
            'roadType': 'float32', # to accept NA values
            'reliability': 'int8',
            'magvar': 'int16',
            'street': 'object'
        },
)

CPU times: user 22.5 ms, sys: 24.5 ms, total: 47.1 ms
Wall time: 79 ms


In [34]:
data = {'schools': schools, 'hospitals': hospitals}
location = "/Users/catherinehayden/WB/cluj/" #where on your system do you want clean.csv?

for name in data: 
    output_path = location + name + ".csv"

    df = pd.DataFrame(data[name])
    gdf = gpd.GeoDataFrame(data[name], geometry='geometry')
    df['lon'] = gdf.geometry.x
    df['lat'] = gdf.geometry.y

    for i in [6,7,8,9,10]:
        df["h" + str(i)] = df.apply(lambda x: h3.geo_to_h3(x["lat"], x["lon"], i), axis=1)
    df.to_csv(output_path, index=False)
